<h1>Stacked Auto Encoders Training 6-15-2016</h1>

Assuming CNN-H1, and the 6 CNNs for the patches have been trained, we can use the weights to get activations for their fc6 layer and train 3 layers of our SAE individually.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu1')

import os
import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Process
from scipy.ndimage.filters import gaussian_filter

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers

# from keras.utils import np_utils

np.random.seed(123456)

Using gpu device 1: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)
Using Theano backend.


In [3]:
cnn_h1_weight_file = 'CNN-H1_epoch-20_activations'
cnn_p1_weight_file = 'CNN-P1_epoch-20_activations'
cnn_p2_weight_file = 'CNN-P2_epoch-20_activations'
cnn_p3_weight_file = 'CNN-P3_epoch-20_activations'
cnn_p4_weight_file = 'CNN-P4_epoch-20_activations'
cnn_p5_weight_file = 'CNN-P5_epoch-20_activations'
cnn_p6_weight_file = 'CNN-P6_epoch-20_activations'
y_train_file = 'y_train_activations'

layer_1_size = 2048
layer_2_size = 1024
layer_3_size = 512

validation_split = .15


In [4]:
def load_activations(file_name):
    return np.load('models/' + file_name + '.npy')

In [5]:
cnn_h1_activations = load_activations(cnn_h1_weight_file)
cnn_p1_activations = load_activations(cnn_p1_weight_file)
cnn_p2_activations = load_activations(cnn_p2_weight_file)
cnn_p3_activations = load_activations(cnn_p3_weight_file)
cnn_p4_activations = load_activations(cnn_p4_weight_file)
cnn_p5_activations = load_activations(cnn_p5_weight_file)
cnn_p6_activations = load_activations(cnn_p6_weight_file)
y_train = load_activations(y_train_file)

In [6]:
def fix_activations(_activations):
    return np.array([activation for activations in _activations for activation in activations])

In [7]:
cnn_h1_activations = fix_activations(cnn_h1_activations)
cnn_p1_activations = fix_activations(cnn_p1_activations)
cnn_p2_activations = fix_activations(cnn_p2_activations)
cnn_p3_activations = fix_activations(cnn_p3_activations)
cnn_p4_activations = fix_activations(cnn_p4_activations)
cnn_p5_activations = fix_activations(cnn_p5_activations)
cnn_p6_activations = fix_activations(cnn_p6_activations)

In [8]:
cnn_h1_activations.shape

(26668, 512)

In [9]:
activations = np.concatenate((
        cnn_h1_activations, 
        cnn_p1_activations,
        cnn_p2_activations,
        cnn_p3_activations,
        cnn_p4_activations,
        cnn_p5_activations,
        cnn_p6_activations), axis=1)

In [10]:
activations.shape

(26668, 3584)

In [11]:
y_train.shape

(26668,)

<h2>Random Noise</h2>

In [59]:
def random_noise(activations, random_threshold, gaussian_scale):
    return np.array([np.array([value + np.random.normal(scale=gaussian_scale) 
                      if np.random.rand() < random_threshold else value
                      for value in activation]) for activation in activations])       

In [63]:

actirandom_noise(activations, 0.05, 1)[0][0]

4.33587
4.33586788177


In [18]:
print activations[0].std()
print activations[0].mean()

5.98296
-0.0329306


In [19]:
print activations[1].std()
print activations[1].mean()

6.41089
-0.0106803


In [21]:
print activations[2].std()
print activations[2].mean()

6.69122
0.0324382


<h2>Auto Encoder</h2>

In [16]:
def get_encoder(X_train, X_test, encoding_dim, nb_epoch=50, regularizer=10e-5):
    assert X_train.shape[1] == X_test.shape[1]
    shape = X_train.shape[1]

    input_img = Input(shape=(shape,))
    encoded = Dense(
        encoding_dim, 
        activation='relu', 
        activity_regularizer=regularizers.activity_l1(regularizer)
    )(input_img)
    decoded = Dense(shape, activation='sigmoid')(encoded)

    autoencoder = Model(input=input_img, output=decoded)
    encoder = Model(input=input_img, output=encoded)

    # sgd = SGD(lr=.01, decay=99.9) #lr=0.01->0.00001
    autoencoder.compile(
        optimizer='adadelta', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])
    
    autoencoder.fit(X_train, X_train,
                nb_epoch=nb_epoch,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))
    return autoencoder, encoder
    

In [17]:
x_train = activations[:int(activations.shape[0] * (1 - validation_split))]
x_test = activations[int(activations.shape[0] * (1 - validation_split)):]
X_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
X_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [19]:
autoencoder1, encoder1 = get_encoder(X_train, X_test, 2048, 30, 10e-3)

Train on 22667 samples, validate on 4001 samples
Epoch 1/30
22667/22667 [==============================] - 4s - loss: -58008.0222 - acc: 0.0472 - val_loss: -65405.0088 - val_acc: 0.0915
Epoch 2/30
22667/22667 [==============================] - 4s - loss: -67501.4108 - acc: 0.1170 - val_loss: -68059.2102 - val_acc: 0.1322
Epoch 3/30
22667/22667 [==============================] - 4s - loss: -69715.2317 - acc: 0.1620 - val_loss: -70155.4506 - val_acc: 0.1750
Epoch 4/30
22667/22667 [==============================] - 4s - loss: -70973.7779 - acc: 0.1938 - val_loss: -71081.0909 - val_acc: 0.2154
Epoch 5/30
22667/22667 [==============================] - 4s - loss: -71885.2786 - acc: 0.2223 - val_loss: -71595.0744 - val_acc: 0.2194
Epoch 6/30
22667/22667 [==============================] - 4s - loss: -72436.4506 - acc: 0.2419 - val_loss: -72164.8902 - val_acc: 0.2444
Epoch 7/30
22667/22667 [==============================] - 4s - loss: -72923.6938 - acc: 0.2591 - val_loss: -72572.0802 - val_acc:

In [20]:
X_train = encoder1.predict(X_train)
X_test = encoder1.predict(X_test)

print X_train.shape
print X_test.shape

/usr/local/lib/python2.7/dist-packages/keras/backend/theano_backend.py:514: UserWarning: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 1 is not part of the computational graph needed to compute the outputs: keras_learning_phase.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.
  **kwargs)


(22667, 2048)
(4001, 2048)


In [21]:
autoencoder2, encoder2 = get_encoder(X_train, X_test, 1024, 50)

Train on 22667 samples, validate on 4001 samples
Epoch 1/50
22667/22667 [==============================] - 2s - loss: 89575.5536 - acc: 4.4117e-05 - val_loss: 88716.9988 - val_acc: 0.0000e+00
Epoch 2/50
22667/22667 [==============================] - 2s - loss: 88642.7791 - acc: 0.0011 - val_loss: 88070.1681 - val_acc: 0.0032
Epoch 3/50
22667/22667 [==============================] - 2s - loss: 87670.1065 - acc: 0.0187 - val_loss: 86770.9366 - val_acc: 0.0607
Epoch 4/50
22667/22667 [==============================] - 2s - loss: 86160.3324 - acc: 0.1035 - val_loss: 85390.1926 - val_acc: 0.1360
Epoch 5/50
22667/22667 [==============================] - 2s - loss: 84872.4734 - acc: 0.1710 - val_loss: 84338.6513 - val_acc: 0.2069
Epoch 6/50
22667/22667 [==============================] - 2s - loss: 83917.2218 - acc: 0.2340 - val_loss: 83606.6558 - val_acc: 0.2707
Epoch 7/50
22667/22667 [==============================] - 2s - loss: 83215.7229 - acc: 0.2858 - val_loss: 83229.7503 - val_acc: 0.312

In [22]:
X_train = encoder2.predict(X_train)
X_test = encoder2.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 1024)
(4001, 1024)


In [23]:
autoencoder3, encoder3 = get_encoder(X_train, X_test, 512, 50)

Train on 22667 samples, validate on 4001 samples
Epoch 1/50
22667/22667 [==============================] - 1s - loss: 14029.9160 - acc: 0.0015 - val_loss: 13265.6123 - val_acc: 0.0027
Epoch 2/50
22667/22667 [==============================] - 1s - loss: 13362.6928 - acc: 0.0108 - val_loss: 12705.4877 - val_acc: 0.0675
Epoch 3/50
22667/22667 [==============================] - 1s - loss: 12744.5180 - acc: 0.2040 - val_loss: 12193.6591 - val_acc: 0.3007
Epoch 4/50
22667/22667 [==============================] - 1s - loss: 12310.4127 - acc: 0.3169 - val_loss: 11904.1355 - val_acc: 0.3799
Epoch 5/50
22667/22667 [==============================] - 1s - loss: 12049.9664 - acc: 0.3601 - val_loss: 11759.3312 - val_acc: 0.3712
Epoch 6/50
22667/22667 [==============================] - 1s - loss: 11878.7187 - acc: 0.3820 - val_loss: 11606.4882 - val_acc: 0.3977
Epoch 7/50
22667/22667 [==============================] - 1s - loss: 11753.1725 - acc: 0.4040 - val_loss: 11515.3912 - val_acc: 0.4374
Epoch 

In [24]:
X_train = encoder3.predict(X_train)
X_test = encoder3.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 512)
(4001, 512)


<h2>Train all together</h2>

In [14]:
input_img = Input(shape=(3584,))
encoded = Dense(2048, activation='relu')(input_img)
encoded = Dense(1024, activation='relu')(encoded)
encoded = Dense(512, activation='relu')(encoded)

decoded = Dense(1024, activation='relu')(encoded)
decoded = Dense(2048, activation='relu')(decoded)
decoded = Dense(3584, activation='sigmoid')(decoded)

autoencoder = Model(input=input_img, output=decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

autoencoder.fit(X_train, X_train,
                nb_epoch=100,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 22667 samples, validate on 4001 samples
Epoch 1/100
22667/22667 [==============================] - 4s - loss: -30.7396 - acc: 0.0057 - val_loss: -38.3032 - val_acc: 0.0122
Epoch 2/100
22667/22667 [==============================] - 4s - loss: -40.9426 - acc: 0.0139 - val_loss: -42.7448 - val_acc: 0.0165
Epoch 3/100
22667/22667 [==============================] - 4s - loss: -45.7900 - acc: 0.0219 - val_loss: -47.3148 - val_acc: 0.0320
Epoch 4/100
22667/22667 [==============================] - 4s - loss: -48.7072 - acc: 0.0340 - val_loss: -49.0509 - val_acc: 0.0430
Epoch 5/100
22667/22667 [==============================] - 4s - loss: -50.5351 - acc: 0.0434 - val_loss: -51.4600 - val_acc: 0.0517
Epoch 6/100
22667/22667 [==============================] - 4s - loss: -52.1692 - acc: 0.0507 - val_loss: -52.2314 - val_acc: 0.0570
Epoch 7/100
22667/22667 [==============================] - 4s - loss: -53.5487 - acc: 0.0566 - val_loss: -53.6521 - val_acc: 0.0740
Epoch 8/100
22667/22667 [==